# Capstone Project - The Battle of the Neighborhoods (Week 1)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)


# Introduction: Business Problem 

In this project we will try to find an **optimal location for a Bar**. Specifically, this report will be targeted to stakeholders interested in **opening a Bar in Toronto, Canada**.

Canada is a country which allows migrants from other countries, as the number of immigrants are getting permanent residence in the Canada, most of them try to open a new business for the survival. As, Toronto is the city with high population, high facilities immigrants try to find places to open a new business in Toronto but strugles which business can be open at which location of the city. To overcome this we will analyze the venues at different Borough by taking the example of Bar business. 

Since there are lots of Bar in Toronto, we will try to detect locations that are not already crowded with Bar. We are also particularly interested in areas with no Bar is in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

# Data

Based on definition of our problem, factors that will influence our decission are:
* number of existing Bar in the neighborhood 
* number of and distance to Bars in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Toronto center will be obtained using **Google Maps API geocoding** of well known Toronto location
* we will be using the url - **https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M** for extracting  data

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
tor_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(tor_url,'lxml')

table = soup.find('table')

column_names= ['Postalcode','Bourough','Neighbourhood']
df = pd.DataFrame(columns = column_names)

In [3]:
table1=""
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

Ignorning cells with 'Not Assigned'

In [4]:
df = df[df['Bourough']!='Not assigned']

In [5]:
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df.head()

,Postalcode,Bourough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
new_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
new_df=new_df.reset_index(drop=False)
new_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [7]:
df_merged = pd.merge(df, new_df, on='Postalcode')

In [8]:
df_merged.drop(['Neighbourhood_joined'],axis=1,inplace=True)

In [9]:
df_merged.drop_duplicates(inplace=True)

In [10]:
df_merged.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

Dataframe Head

In [11]:
df_merged.head(12)

,Postalcode,Bourough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Dataframe Shape

In [12]:
df_merged.shape

(103, 3)

In [13]:
#Using dataframes from question 1
def get_geocode(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [14]:
loc_df=pd.read_csv('https://cocl.us/Geospatial_data')

In [15]:
loc_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
loc_merged = pd.merge(loc_df, df_merged, on='Postalcode')

In [16]:

loc_data=loc_merged[['Postalcode','Bourough','Neighbourhood','Latitude','Longitude']]

In [17]:
loc_data.head()

,Postalcode,Bourough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
#import libaries
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [19]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))


map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

The coordinates of Toronto are 43.6534817, -79.3839347.


In [20]:
for lat, lng, borough, Neighbourhood in zip(loc_data['Latitude'], loc_data['Longitude'], loc_data['Bourough'], loc_data['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Foursquare

In [21]:
#setting the radius and limit 
radius=500
LIMIT=100

In [22]:
CLIENT_ID = 'KNKMTD5IXBLJQLKBOUFAENFNBG1EABHQ1CCQ15L2WFVH4W3V' # your Foursquare ID
CLIENT_SECRET = 'OECNZ22ORJO3OUSAQPVQC2255AENFWMNYDOFRFH2NTBYJUBH' # your Foursquare Secret
VERSION = '20200507'

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=loc_data['Neighbourhood'],
                                   latitudes=loc_data['Latitude'],
                                   longitudes=loc_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

### Venues head

In [25]:
toronto_venues.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
9,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
